# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response_pipeline.db')
df = pd.read_sql_table('disaster_response_pipeline', con=engine)  
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [3]:
# check the columns of dataset df
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# check the columns of variable X
X.head(1)

0    Weather update - a cold front from Cuba that c...
Name: message, dtype: object

In [5]:
# check the columns of variable Y
y.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [6]:
# define tokenize function
def tokenize(text):
    """
    Function:   Tokenization function to process text data.
    Args:       text
    Return:     clean_tokens
    """
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# build ML pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(criterion='gini', n_estimators=50, max_depth=2, random_state=1)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x11b8c4ca0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=2,
                                                                        n_estimators=50,
                                                                        random_state=1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# get number or columns of y
columns_count = len(y.columns)

In [10]:
# define predict test data
y_pred = pipeline.predict(X_test)
y_true = y_test.values

In [11]:
# report f1 score precision
def ml_model(y_true, y_pred):
    """
    Function:   Reporting f1-score precision data.
    Args:       y_true, y_pred
    Return:     Classification report for each column.
    """
    for i in range(columns_count):
        print('Report: ' + y_test.columns[i],'\n',
              classification_report(y_true[:, i],y_pred[:, i], output_dict=False, zero_division=1))
        

In [12]:
# use model
ml_model(y_true, y_pred)

Report: related 
               precision    recall  f1-score   support

         0.0       1.00      0.00      0.00       851
         1.0       0.66      1.00      0.79      1676
         2.0       1.00      0.00      0.00        29

    accuracy                           0.66      2556
   macro avg       0.89      0.33      0.26      2556
weighted avg       0.77      0.66      0.52      2556

Report: request 
               precision    recall  f1-score   support

         0.0       0.65      1.00      0.79      1658
         1.0       1.00      0.00      0.00       898

    accuracy                           0.65      2556
   macro avg       0.82      0.50      0.39      2556
weighted avg       0.77      0.65      0.51      2556

Report: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2556

    accuracy                           1.00      2556
   macro avg       1.00      1.00      1.00      2556
weighted avg       1.0

Report: storm 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      2469
         1.0       1.00      0.00      0.00        87

    accuracy                           0.97      2556
   macro avg       0.98      0.50      0.49      2556
weighted avg       0.97      0.97      0.95      2556

Report: fire 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2548
         1.0       1.00      0.00      0.00         8

    accuracy                           1.00      2556
   macro avg       1.00      0.50      0.50      2556
weighted avg       1.00      1.00      1.00      2556

Report: earthquake 
               precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      2349
         1.0       1.00      0.00      0.00       207

    accuracy                           0.92      2556
   macro avg       0.96      0.50      0.48      2556
weighted avg       0.9

None

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
# define parameters of tuned model
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__min_samples_split': [2, 2]
    }

# use sklearn grid search to create improved model
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=5, n_jobs=-1)

# train cv
cv.fit(X_train, y_train)


Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    7.7s remaining:   12.9s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:   13.0s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   14.2s finished


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x11b8c4ca0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=2,
                                                                                               n_estimators=50,
                                                                                               random_state=1)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__min_samples_split': [1, 2],
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=5)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
# define predict test data of tuned model
y_pred = cv.predict(X_test)
ml_model(y_true, y_pred)

Report: related 
               precision    recall  f1-score   support

         0.0       1.00      0.00      0.00       851
         1.0       0.66      1.00      0.79      1676
         2.0       1.00      0.00      0.00        29

    accuracy                           0.66      2556
   macro avg       0.89      0.33      0.26      2556
weighted avg       0.77      0.66      0.52      2556

Report: request 
               precision    recall  f1-score   support

         0.0       0.65      1.00      0.79      1658
         1.0       1.00      0.00      0.00       898

    accuracy                           0.65      2556
   macro avg       0.82      0.50      0.39      2556
weighted avg       0.77      0.65      0.51      2556

Report: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2556

    accuracy                           1.00      2556
   macro avg       1.00      1.00      1.00      2556
weighted avg       1.0

Report: storm 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      2469
         1.0       1.00      0.00      0.00        87

    accuracy                           0.97      2556
   macro avg       0.98      0.50      0.49      2556
weighted avg       0.97      0.97      0.95      2556

Report: fire 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2548
         1.0       1.00      0.00      0.00         8

    accuracy                           1.00      2556
   macro avg       1.00      0.50      0.50      2556
weighted avg       1.00      1.00      1.00      2556

Report: earthquake 
               precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      2349
         1.0       1.00      0.00      0.00       207

    accuracy                           0.92      2556
   macro avg       0.96      0.50      0.48      2556
weighted avg       0.9

In [15]:
# show best parameters of ML model
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'vect__ngram_range': (1, 1)}

In [16]:
# show best score of ML model
cv.best_score_

0.18025293347274807

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# build ML imporved ML pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('moc_knc', MultiOutputClassifier(KNeighborsClassifier()))
])

In [18]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train improved pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x11b8c4ca0>)),
                ('tfidf', TfidfTransformer()),
                ('moc_knc',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [19]:
# define predict test data
y_pred = pipeline.predict(X_test)
y_true = y_test.values

In [20]:
# report f1 score precision of improved model
ml_model(y_true, y_pred)

Report: related 
               precision    recall  f1-score   support

         0.0       0.36      0.26      0.30       838
         1.0       0.67      0.77      0.71      1686
         2.0       0.33      0.03      0.06        32

    accuracy                           0.59      2556
   macro avg       0.45      0.35      0.36      2556
weighted avg       0.56      0.59      0.57      2556

Report: request 
               precision    recall  f1-score   support

         0.0       0.66      0.94      0.77      1669
         1.0       0.39      0.08      0.13       887

    accuracy                           0.64      2556
   macro avg       0.52      0.51      0.45      2556
weighted avg       0.56      0.64      0.55      2556

Report: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2553
         1.0       1.00      0.00      0.00         3

    accuracy                           1.00      2556
   macro avg       1.0

Report: storm 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.99      2489
         1.0       0.00      0.00      0.00        67

    accuracy                           0.97      2556
   macro avg       0.49      0.50      0.49      2556
weighted avg       0.95      0.97      0.96      2556

Report: fire 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2548
         1.0       1.00      0.00      0.00         8

    accuracy                           1.00      2556
   macro avg       1.00      0.50      0.50      2556
weighted avg       1.00      1.00      1.00      2556

Report: earthquake 
               precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      2347
         1.0       0.00      0.00      0.00       209

    accuracy                           0.92      2556
   macro avg       0.46      0.50      0.48      2556
weighted avg       0.8

### 9. Export your model as a pickle file

In [21]:
# export model to pickle file
pickle.dump(cv,open('ml_model.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.